<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/Authorship-Identification/blob/main/topic_anlalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling

In [ ]:
!pip install -qU bertopic

## Embed Articles

In [ ]:
from sentence_transformers import SentenceTransformer
import pickle

In [ ]:
model_id = "Omartificial-Intelligence-Space/GATE-AraBert-v1"
embedding_model = SentenceTransformer(model_id, device="cuda:0")

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
articles_embeddings = embedding_model.encode(train_data['text_in_author_style'].values, show_progress_bar=True, batch_size=64)

print(articles_embeddings.shape)

Batches:   0%|          | 0/513 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Store data and embeddings
with open("bertarabic_embeddings_splitted.pkl", "wb") as fOut:
  pickle.dump({'embeddings': articles_embeddings},
              fOut,
              protocol = pickle.HIGHEST_PROTOCOL)

In [ ]:
# Generate embeddings in batches to avoid OutOfMemoryError
batch_size = 16 # Reduced batch size
# Use Matryoshka embeddings by specifying the output_dimension
# Replace 256 with your desired dimension
articles_embeddings = embedding_model.encode(train_data['text_in_author_style'].values, show_progress_bar=True, batch_size=batch_size, output_dimension=256)
print(articles_embeddings.shape)